# L5: Automate Event Planning

In this lesson, you will learn more about Tasks.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
from IPython.display import Markdown
import os
from time import time
import json
from pprint import pprint

# import crewai   # Version: 0.30.11
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool

from crewai_tools import (
    SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool,
    DirectoryReadTool, FileReadTool, BaseTool
)
from pydantic import BaseModel

from api_key_store import ApiKeyStore
from helper import convert_to_filename

# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [ ]:
s = ApiKeyStore()

os.environ["OPENAI_API_KEY"] = s.get_api_key(provider="OPENAI")  # os.environ["API_KEY_OPENAI"]
os.environ["SERPER_API_KEY"] = s.get_api_key(provider="SerpApi")  # os.environ["API_KEY_SERPAPI"]

os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'   # gpt-4 was default, throws rate-limit error 

In [4]:
file_data_json = "venue_details.json"
file_out_markdown = "marketing_report.md"

## crewAI Tools

In [5]:
# Initialize the tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [6]:
# Agent 1: Venue Coordinator
venue_coordinator = Agent(
    role="Venue Coordinator",
    goal="Identify and book an appropriate venue  based on event requirements",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory="""
        "With a keen sense of space and "
        "understanding of event logistics, "
        "you excel at finding and securing "
        "the perfect venue that fits the event's theme, "
        "size, and budget constraints."
    """.replace('"',''),
)

In [7]:
 # Agent 2: Logistics Manager
logistics_manager = Agent(
    role='Logistics Manager',
    goal="Manage all logistics for the event including catering and equipmen",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory="""
        "Organized and detail-oriented, "
        "you ensure that every logistical aspect of the event "
        "from catering to equipment setup "
        "is flawlessly executed to create a seamless experience."
    """.replace('"',''),
)

In [8]:
# Agent 3: Marketing and Communications Agent
marketing_communications_agent = Agent(
    role="Marketing and Communications Agent",
    goal="Effectively market the event and  communicate with participants",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory="""
        "Creative and communicative, "
        "you craft compelling messages and "
        "engage with potential attendees "
        "to maximize event exposure and participation."
    """.replace('"',''),
)

## Creating Venue Pydantic Object

- Create a class `VenueDetails` using [Pydantic BaseModel](https://docs.pydantic.dev/latest/api/base_model/).
- Agents will populate this object with information about different venues by creating different instances of it.

In [9]:

# Define a Pydantic model for venue details 
# (demonstrating Output as Pydantic)
class VenueDetails(BaseModel):
    name: str
    address: str
    capacity: int
    booking_status: str

## Creating Tasks
- By using `output_json`, you can specify the structure of the output you want.
- By using `output_file`, you can get your output in a file.
- By setting `human_input=True`, the task will ask for human feedback (whether you like the results or not) before finalising it.

In [10]:
venue_task = Task(
    description="Find a venue in {event_city} that meets criteria for {event_topic}.",
    expected_output="All the details of a specifically chosen venue you found to accommodate the event.",
    human_input=True,
    output_json=VenueDetails,
    output_file=file_data_json,   # Outputs the venue details as a JSON file
    agent=venue_coordinator
)

- By setting `async_execution=True`, it means the task can run in parallel with the tasks which come after it.

In [11]:
logistics_task = Task(
    description="Coordinate catering and equipment for an event with {expected_participants} participants on {tentative_date}.",
    expected_output="Confirmation of all logistics arrangements including catering and equipment setup.",
    human_input=True,
    async_execution=True,
    agent=logistics_manager
)

In [12]:
marketing_task = Task(
    description="Promote the {event_topic} aiming to engage at least {expected_participants} potential attendees.",
    expected_output="Report on marketing activities and attendee engagement formatted as markdown.",
    async_execution=True,
    output_file=file_out_markdown,  # Outputs the report as a text file
    agent=marketing_communications_agent
)

## Creating the Crew

**Note**: Since you set `async_execution=True` for `logistics_task` and `marketing_task` tasks, now the order for them does not matter in the `tasks` list.

In [13]:
# Define the crew with agents and tasks
event_management_crew = Crew(
    agents=[venue_coordinator, 
            logistics_manager, 
            marketing_communications_agent],
    
    tasks=[venue_task, 
           logistics_task, 
           marketing_task],
    
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [14]:
event_details = {
    'event_topic': "Tech Innovation Conference",
    'event_description': "A gathering of tech innovators and industry leaders to explore future technologies.",
    'event_city': "San Francisco",
    'tentative_date': "2024-09-15",
    'expected_participants': 500,
    'budget': 20000,
    'venue_type': "Conference Hall"
}

**Note 1**: 
- LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

**Note 2**: 
- Since you set `human_input=True` for some tasks, the execution will ask for your input before it finishes running.
- When it asks for feedback, use your mouse pointer to first click in the text box before typing anything.

In [15]:
ts_start = time()
result = event_management_crew.kickoff(inputs=event_details)
ts_stop = time()

 [DEBUG]: == Working Agent: Venue Coordinator
 [INFO]: == Starting Task: Find a venue in San Francisco that meets criteria for Tech Innovation Conference.


> Entering new CrewAgentExecutor chain...
I need to start by researching potential venues in San Francisco that can accommodate the Tech Innovation Conference.

Action:
Search the internet
Action Input:
{
  "search_query": "tech conference venues in San Francisco"
}
 


Search results: Title: Top 12 tech conferences in San Francisco for 2024 - TravelPerk
Link: https://www.travelperk.com/blog/top-tech-conferences-in-san-francisco/
Snippet: Running for over twenty years, Dreamforce -- Salesforce's flagship annual conference -- is widely known as one of the biggest tech conferences ...
---
Title: Best Science & Tech Conference in San Francisco, CA - Eventbrite
Link: https://www.eventbrite.com/d/ca--san-francisco/science-and-tech--conferences/
Snippet: Science & tech events in San Francisco, CA ; GenAI Summit San Francisco 2024. Wednes

This is the agent final answer: After researching potential venues in San Francisco for the Tech Innovation Conference, I have found the Main Hall at Trellis in South Beach, San Francisco, CA to be a suitable venue. The Main Hall can accommodate 150 guests seated or 200 standing with a minimum spend of $1500 per session. The venue offers a versatile space with rustic charm, brick walls, redwood rafters, and curated art and furniture. It is an inviting space perfect for hosting a special event like the Tech Innovation Conference. Additionally, the venue provides amenities such as Wi-Fi, projectors, screens, comfortable seating options, and a PA system, making it ideal for a conference setting. Overall, the Main Hall at Trellis at South Beach, San Francisco, CA, meets the criteria for hosting the Tech Innovation Conference with its capacity, amenities, and ambiance.
Please provide a feedback:  great, proceed with your recommendation


Final Answer:
After researching potential venues in San Francisco for the Tech Innovation Conference, I have found the Main Hall at Trellis in South Beach, San Francisco, CA to be a suitable venue. The Main Hall can accommodate 150 guests seated or 200 standing with a minimum spend of $1500 per session. The venue offers a versatile space with rustic charm, brick walls, redwood rafters, and curated art and furniture. It is an inviting space perfect for hosting a special event like the Tech Innovation Conference. Additionally, the venue provides amenities such as Wi-Fi, projectors, screens, comfortable seating options, and a PA system, making it ideal for a conference setting. Overall, the Main Hall at Trellis at South Beach, San Francisco, CA, meets the criteria for hosting the Tech Innovation Conference with its capacity, amenities, and ambiance.

> Finished chain.
 [DEBUG]: == [Venue Coordinator] Task output: {
  "name": "Main Hall at Trellis",
  "address": "South Beach, San Francisco

Exception in thread Thread-6 (_execute):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/home/papagame/.local/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/home/papagame/.local/lib/python3.10/site-packages/crewai/task.py", line 192, in _execute
    result = agent.execute_task(
  File "/home/papagame/.local/lib/python3.10/site-packages/crewai/agent.py", line 236, in execute_task
    result = self.agent_executor.invoke(
  File "/home/papagame/.local/lib/python3.10/site-packages/langchain/chains/base.py", line 163, in invoke
    raise e
  File "/home/papagame/.local/lib/python3.10/site-packages/langchain/chains/base.py", line 153, in invoke
    self._call(inputs, run_manager=run_manager)
  File "/home/papagame/.local/lib/python3.

Thought: I need to start promoting the Tech Innovation Conference and engaging potential attendees to meet the goal of at least 500 participants.

Action: Search the internet
Action Input: {"search_query": "Tech Innovation Conference marketing strategies"}
 


Search results: Title: Innovative Event Marketing Strategies to Boost Corporate and Social ...
Link: https://www.linkedin.com/pulse/innovative-event-marketing-strategies-boost-corporate-b70rc
Snippet: These strategies are designed to attract, entertain, and leave a lasting impression on attendees even after the event is over. From intriguing ...
---
Title: 10 Effective Conference Marketing Ideas for 2024 | InEvent
Link: https://inevent.com/blog/others/conference-marketing-ideas.html
Snippet: 1. Focus on Target Audience and Personalization · 2. Leverage Digital Marketing for Conference Advertising · 3. Create a Stunning Event Website · 4 ...
---
Title: 2024 Digital Marketing Strategies Conference | Austin, TX
Link: https://digital

- Display the generated `venue_details.json` file.

In [18]:
print(f"\n\nElapsed time: {(ts_stop-ts_start):.2f} sec")



Elapsed time: 75.50 sec


In [16]:
with open(file_data_json) as f:
   data = json.load(f)

pprint(data)

{'address': 'South Beach, San Francisco, CA',
 'booking_status': '$1500 minimum spend per session',
 'capacity': 150,
 'name': 'Main Hall at Trellis'}


- Display the generated `marketing_report.md` file.

**Note**: After `kickoff` execution has successfully ran, wait an extra 45 seconds for the `marketing_report.md` file to be generated. If you try to run the code below before the file has been generated, your output would look like:

```
marketing_report.md
```

If you see this output, wait some more and than try again.

In [28]:
with open(file_out_markdown) as f:
    md_str = f.read()

Markdown(md_str)

## Marketing Activities and Attendee Engagement Report

### Marketing Strategies:
1. **Focus on Target Audience and Personalization**: Tailor marketing messages to specific audience segments to increase engagement.
2. **Leverage Digital Marketing for Conference Advertising**: Utilize digital channels such as social media, email, and online ads to reach a wider audience.
3. **Create a Stunning Event Website**: Develop an engaging and informative website that showcases the conference details and encourages registration.
4. **Identify Your Target Audience**: Understand the demographics and preferences of potential attendees to create targeted marketing campaigns.
5. **Differentiate Your Audiences**: Tailor marketing strategies based on the different audience segments to maximize interest and participation.
6. **Build a Brand for Your Event**: Establish a strong brand identity for the conference to increase recognition and attract more attendees.
7. **Utilize Social Media Platforms**: Engage with potential attendees on social media platforms to share event updates and create buzz.
8. **Partner with Influencers**: Collaborate with industry influencers to reach a larger audience and increase event visibility.
9. **Offer Early Bird Discounts**: Encourage early registration by offering discounted rates for attendees who sign up in advance.
10. **Create Engaging Video Content**: Develop video content that highlights the value and excitement of the conference to attract attendees.

### Attendee Engagement Strategies:
1. **Organize Interactive Sessions**: Plan interactive sessions like networking games, Q&As, and polls to keep attendees engaged.
2. **Implement Technology-Driven Engagement**: Use event technology to enhance attendee experiences and showcase innovation.
3. **Promote Knowledge Sharing**: Create opportunities for attendees to share knowledge and insights with each other during the conference.
4. **Encourage Meaningful Interactions**: Foster meaningful interactions between attendees to build connections and enhance engagement.
5. **Utilize Wearables and Tech Gadgets**: Incorporate high-tech wearables and gadgets to enhance the attendee experience and encourage participation.
6. **Offer Surveys and Feedback Opportunities**: Collect feedback from attendees through surveys and polls to improve future events.
7. **Utilize Social Media Platforms for Engagement**: Engage with attendees on social media before, during, and after the conference to maintain interest and interaction.
8. **Create Challenges and Scavenger Hunts**: Organize challenges and scavenger hunts that encourage attendees to explore different aspects of the event.

By implementing these marketing and engagement strategies, the Tech Innovation Conference aims to attract a large audience of over 500 participants and provide them with a valuable and engaging experience.